In [2]:
import os
import pickle
import shutil
from pathlib import Path
from synplan.utils.loading import download_all_data

from CGRtools import smiles
from IPython.display import SVG, display
from synplan.utils.visualisation import get_route_svg
from synplan.mcts.tree import Tree
from synplan.mcts.evaluation import ValueNetworkFunction
from synplan.mcts.expansion import PolicyNetworkFunction
from synplan.utils.config import PolicyNetworkConfig
from synplan.utils.config import TreeConfig
from synplan.utils.loading import load_reaction_rules, load_building_blocks

import pandas as pd
from collections import Counter, defaultdict
from time import time

In [3]:
tree_config = TreeConfig(
    search_strategy="expansion_first",
    evaluation_type="rollout",
    max_iterations=300,
    max_time=600000,
    max_depth=9,
    min_mol_size=1,
    init_node_value=0.5,
    ucb_type="uct",
    c_ucb=0.1,
)

# input data
policy_network = "training_hybrid/ranking_policy_network/policy_network.ckpt"
reaction_rules = load_reaction_rules("training_hybrid/reaction_rules.pickle")
building_blocks = load_building_blocks("synplan_data/building_blocks/building_blocks_em_sa_ln.smi", standardize=False)

policy_config = PolicyNetworkConfig(weights_path=policy_network)
policy_function = PolicyNetworkFunction(policy_config=policy_config)

In [4]:
target_list = list(pd.read_csv("synplan_data/benchmarks/sascore/targets_with_sascore_1.5_2.5.smi", header=None)[0])

target = smiles(target_list[2])
target.canonicalize()
target.clean2d()
target

In [5]:
tree = Tree(
    target=target,
    config=tree_config,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    expansion_function=policy_function,
    evaluation_function=None,
)
_ = list(tree)
print(tree.winning_nodes)

  0%|          | 0/300 [00:00<?, ?it/s]

[11, 18, 25, 26, 26, 32, 33, 34, 35, 38, 39, 41, 42, 32, 33, 34, 35, 43, 43, 49, 50, 51, 52, 55, 56, 58, 59, 60, 60, 66, 67, 68, 69, 72, 73, 75, 76, 80, 82, 87, 88, 89, 90, 96, 97, 98, 96, 106, 107, 107, 113, 114, 115, 116, 119, 120, 122, 123, 124, 124, 130, 131, 132, 133, 136, 137, 139, 140, 141, 146, 97, 186, 187, 187, 193, 194, 195, 196, 199, 200, 202, 203, 204, 205, 206, 208, 209, 214, 215, 204, 205, 206, 226, 227, 216, 217, 218, 220, 221, 233, 234, 234, 240, 241, 242, 243, 246, 247, 249, 250, 251, 251, 257, 258, 259, 260, 263, 264, 266, 267, 330, 18, 334, 335, 336, 338, 339, 344, 345, 334, 335, 336, 350, 356, 356, 368, 378, 386, 394, 402, 406, 410, 419, 420, 437, 451, 443, 446, 456, 460, 466, 468, 469, 470, 466, 480, 476, 480, 25, 487, 487, 496, 497, 505, 512, 517, 509, 512, 532, 532, 544, 545, 547, 548, 538, 539, 540, 541, 544, 545, 549, 554, 560, 639, 650, 651, 651, 657, 658, 659, 660, 663, 664, 666, 667, 668, 668, 674, 675, 676, 677, 680, 681, 683, 684, 233, 685, 685, 694, 695,

In [7]:
from IPython.display import SVG, display
from synplan.utils.visualisation import get_route_svg

for n, node_id in enumerate(tree.winning_nodes):
    print(
        f"-------- Path starts from node #{node_id} with total route score {tree.route_score(node_id)} --------"
    )
    display(SVG(get_route_svg(tree, node_id)))
    if n == 3:
        break

-------- Path starts from node #11 with total route score 0.058340408293255853 --------


-------- Path starts from node #18 with total route score 0.17800884620648738 --------


-------- Path starts from node #25 with total route score 0.14066832557403086 --------


-------- Path starts from node #26 with total route score 0.11968020591149454 --------
